In [19]:
# setup
from IPython.core.display import display,HTML
display(HTML('<style>.prompt{width: 0px; min-width: 0px; visibility: collapse}</style>'))
display(HTML(open('../rise.css').read()))

# imports
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
sns.set(style="whitegrid", font_scale=1.5, rc={'figure.figsize':(12, 6)})


/tmp/ipykernel_639716/1251851185.py:2: DeprecationWarning: Importing display from IPython.core.display is deprecated since IPython 7.14, please import from IPython display
  from IPython.core.display import display,HTML


# CMPS 2200
# Introduction to Algorithms

<h3> Last Time</h3>

We introduce the Map/Filter/Reduce paradigm and explained the functions that inspire the name. Some of this is included in Chapter 5 of the textbook "Parallel and Sequential Algorithms" under the name "sequences."

Here is a larger list of functions that fit into the paradigm. We assume the items in the list all have the same type. For this list, we assume that each function that is an agument to a map/filter/reduce function can be applied in constant time. We assume the input list has length $n$.
1. Map:
    - Expects a list and a function
    - Returns a new list by applying the function to each item.
    - Work: $\Theta(n)$
    - Span: $\Theta(1)$
    - Example: $Map([1,5,6,3], x+2) = [3,7,8,5]$
2. Filter:
    - Expects a list and a function that returns a Boolean.
    - Returns a new list by removing the items where the function evaluates to False.
    - Work: $\Theta(n)$
    - Span: $\Theta(\log(n))$
    - Example: $Filter([1,5,6,3], x\%2==1) = [1,5,3]$
3. Reduce:
    - Expects a list and a binary associative function whose output type is the same as the elements of the list.
    - Returns the result of applying the binary associative function until only $1$ item remains.
    - Work: $\Theta(n)$
    - Span: $\Theta(\log(n))$
    - Example: $Reduce([1,5,6,3], x+y) = 15$
4. Scan (Accumulate):
    - Expects a list (my_list) and a binary associative function (f) whose output type is the same as the elements of the list.
    - Returns a list of applications of the binary associative function to the prefixes of the list.
    - Can be implemented with Reduce: Returns [reduce(my_list[:i],f) for i in range(len(my_list)+1)]
    - Work $\Theta(n)$
    - Span $\Theta(\log(n))$
    - Example: $Scan([1,5,6,3], x+y) = [1,6,12,15]$
5. Zip:
    - Expects two lists. We assume both are length $n$.
    - Returns a new list whose $i^{th}$ element is the pair of $i^{th}$ elements of the input lists.
    - If the lists are different lengths, then Zip first truncates the longer list so that they have the same length.
    - Work $\Theta(n)$
    - Span $\Theta(1)$
    - Example: $Zip([1,5,6,3], [8,7,1,4]) = [(1,8),(5,7),(6,1),(3,4)]$
6. Enumerate:
    - Expects a list. 
    - Returns a list of pairs, where the first pair is the index.
    - Can be implemented with Zip(my_list, range(len(my_list))).
    - Work: $\Theta(n)$
    - Span: $\Theta(\log(n))$
    - Example: $Enumerate([8,7,1,4]) = [(0,8), (1,7),(2,1),(3,4)]$
7. Collect (Groupby):
    - Expects a list of pairs.
    - Returns a list of pairs. The second element of each pair is a list.
    - The first coordinates of the return pairs are the unique first coordinates of my_list.
    - The second coordinates are the list of second coordinates that are associated with the first coordinate.
    - Example: $Collect([(1,``a"),(5,``b"),(1,``c"),(5,``d")]) = [(1,[``a",``c"]), (2,[``b",``d"])]$
8. Chain: 
    - Expects two lists.
    - Returns the concatenation of both lists.
    - Work: $\Theta(1)$
    - Span: $\Theta(1)$
    - Example: $Chain([8,7,1,4], [8,4,5]) = [8,7,1,4,8,4,5]$
9. Flatten:
    - Expects a list of lists.
    - Returns a list by concatenating all of the given lists.
    - Can be implemented with Chain and Reduce.
    - Work: $\Theta(n)$
    - Span: $\Theta(\log(n))$
    - Example: $Flatten([[8,7,1,4],[8,4,5],[6,6,5]]) = [8,7,1,4,8,4,5,6,6,5]$
10. Length:
    - Expects a list
    - Returns a natural number, the length of the list.
    - Work: $O(1)$
    - Span: $O(1)$
    - Example: $length([1,6,7])=3$
    - Note: We assume the list data structure stores its length. This is [true](https://www.geeksforgeeks.org/internal-working-of-list-in-python/) for lists in python.

There are other higher-order functions that operate on lists. I don't have a complete list, and different sources seem to have different standard functions and different assumptions as to what constitutes a list. For example, the textbook does not mention zip. 

By the "map/filter/Reduce" paradigm, we basically mean the collection of functions on lists that do not refer specifically to looping through the list.

## Scan

To analyze the span of the map/filter/reduce functions listed above, we analyze Scan and explain that Scan can be used as a key step to implement the other functions.

$Scan:(\alpha \times \alpha \rightarrow \alpha) (id : \alpha) (a : \mathbb{S}_\alpha) : (S_\alpha * \alpha)$


Input of Scan is:
- $f$: an associative binary function
- $a$ is the list
- $id$ is the **left identity** of $f$ $\:\: \equiv \:\:$ $f(id, x) = x$ for all $x \in \alpha$

Returns:
  - a value of type $S_\alpha$, the sequence of repeated applications of $f$ to the prefixes of $a$


Here, $f$ can be any associative binary function with the correct types. Some useful examples:
- Sum (We will explain most things in terms of sum)
- Max

The textbook has Scan return 2 things, a list and a value. Essentially we just add the value into the list to have a single output.

In [30]:
def reduce(f, id_, a):
    if len(a) == 0:
        return id_
    elif len(a) == 1:
        return a[0]
    else:
        # can call these in parallel
        return f(reduce(f, id_, a[:len(a)//2]),
                  reduce(f, id_, a[len(a)//2:]))
        
def add(x, y):
    return x + y

def scan(f, id_, a):
    """
    This is a horribly inefficient implementation of scan
    only to understand what it does.
    We'll discuss how to make it more efficient later.
    """
    return [reduce(f, id_, a[:i]) for i in range(len(a)+1)]

scan(add, 0, [2,1,3,2,2,5,4,1])

[0, 2, 3, 6, 8, 10, 15, 19, 20]

`scan` is sometimes called `prefix sum`, as in the previous example it computes the sum of every prefix of a list. But it doesn't necessarily need to be applied with summation.

In [21]:
# what does this do?
import math
def gt(x,y):
    return x if x > y else y

scan(gt, -math.inf, [10,4,5,12,3,16,5]) 

([10, 10, 10, 12, 12, 16, 16], 16)

e.g., recall **Rightmost Positive**

> Given a sequence of integers $a$, for each element in $a$ find the rightmost positive number to its left.

E.g., 

$rpos \: \langle 1, 0, -1, 2, 3, 0, -5, 7 \rangle \Rightarrow \langle -\infty, 1, 1, 1, 2, 3, 3, 3 \rangle$

We solved with `iterate`, but we can also solve with `scan`.

In [22]:
def select_positive(last_positive_value, next_element):
    """
    Params:
      last_positive_value...the last positive value seen
      next_element..........next element from input list
      
    Returns:
      the element to be remembered going forward
    """
    if next_element > 0:        # remember this new value
        return next_element
    else:                       # reuse the old value
        return last_positive_value
    
scan(select_positive, -math.inf, [1,0,-1,2,3,0,-5,7])

([1, 1, 1, 2, 3, 3, 3, 7], 7)


But, because our `scan` implementation is currently slow, this doesn't gain us anything.

Surprisingly, we can reduce the work and span of scan, even though it looks "hopelessly serial."

```python
def scan(f, id_, a):
    """
    This is a horribly inefficient implementation of scan
    only to understand what it does.
    We'll discuss how to make it more efficient later.
    """
    return [reduce(f, id_, a[:i]) for i in range(len(a)+1)]

```

## Improving Scan

We will look at how to improve `scan` approach using `prefix_sum` as an example.

$\text{prefix\_sum}([2,1,3,2,2,5,4,1]) \rightarrow ([0, 2, 3, 6, 8, 10, 15, 19, 20])$


Divide and conquer works, but is not the most efficient.

TODO: explain details. See Textbook.



Instead, we use an idea called **contraction** that is like divide and conquer, but doesn't require subproblems to be independent. Yet it still allows some parallelism.


**Key observation:**

Given input $[2,1,3,2,2,5,4,1]$ we can compute pairwise addition on each adjacent pairs of numbers:



$[2,1,3,2,2,5,4,1] \rightarrow$

$[(2+1), (3+2), (2+5), (4+1)] \rightarrow$

$[3, 5, 7, 5]$

<br>


These four additions can be done in parallel.

> Contraction:
>
> $[2,1,3,2,2,5,4,1] \rightarrow [(2+1), (3+2), (2+5), (4+1)] \rightarrow \pmb{[3, 5, 7, 5]}$

This is a *partial* output. How do we modify this to get the final output?
<br>



If we run prefix sum on this, we get:

$\text{prefix\_sum}([3, 5, 7, 5]) \rightarrow ([0, 3, 8, 15, 20])$



We want to end up with:

$\text{prefix\_sum}([2,1,3,2,2,5,4,1]) \rightarrow ([\mathbf{0}, 2, \mathbf{3}, 6, \mathbf{8}, 10, \mathbf{15}, 19, 20])$



How can we combine this partial solution with the original input $[2,1,3,2,2,5,4,1]$ to get the right answer?


![figures/scan.png](figures/scan.png)

> Sum together the partial output at position $i$ with the original input at $i+1$.

In [23]:
num_compositions = 0
def fastscan(f, id_, a):
    '''Input: f ....a binary, associative function.
              id_ ....The left identity of the function
              a ......A list, all of the same type.
              f must be a function of type l x r -> l where l is the type of id_, r is the type of elements of a.
        Returns:
            A list of reductions by f to the prefixes of a. When f is +, this amounts to partial sums of a.       
    '''
    # base cases
    if len(a) == 0:
        return [id_]
    elif len(a) == 1:
        return [id_, a[0]]
    else:
        # compute the "partial solution" by
        # applying f to each adjacent pair of numbers 
        # e.g., [2, 1, 3, 2, 2, 5, 4, 1] -> [3, 5, 7, 5]
        # this can be done in parallel

        partial_solution = [f(a[i], a[i+1]) for i in range(len(a))[:-1:2]] #i runs through every-other element, except the last to avoid IndexError.
        if len(a)%2==1:
            partial_solution.append(a[-1])#If the length is odd, append the last number.


        # recursively apply fastscan to the partial solution
        partial_output = fastscan(f, id_, partial_solution)
        
        # combine partial_output with input to get desired output
        ret = (
            [partial_output[i//2] if i%2==0 else   # use partial output
             f(partial_output[i//2], a[i-1])       # combine partial output with next value
             for i in range(len(a)+1)]
        )
        return ret

In [31]:
def add(x,y):
    return x + y

print( fastscan(add, 0, [2,1,3,2,2,5,4,1,0])) 
print( scan(add, 0, [2,1,3,2,2,5,4,1,0]) )

[0, 2, 3, 6, 8, 10, 15, 19, 20, 20]
[0, 2, 3, 6, 8, 10, 15, 19, 20, 20]


In [26]:
# also works for other operators.
fastscan(select_positive, -math.inf, [1,0,-1,2,3,0,-5,7])

[-inf, 1, 1, 1, 2, 3, 3, 3, 7]

### Analysis of the Work of `scan` 

Assume that function `f` is constant time. Then

```python
    subproblem = [f(a[i], a[i+1]) for i in range(len(a))[::2]]
```


takes $O(n)$ time. And


```python
        ret = (
            [partial_output[i//2] if i%2==0 else
             f(partial_output[i//2], a[i-1])  
             for i in range(len(a)+1)]
        )
```

takes $O(n)$ time. Therefore,

```python
    partial_output = fastscan(f, id_, subproblem)
```


reduces problem by half on each recursive call,

but there is only one recursive call, instead of two for, e.g., `merge sort`. So the work of fast scan is



$$W(n) = W(n/2) + n$$

$$W(n) \in O(n)$$

This is the same amount of work required to calculate Scan directly by sequentially through $a$. When the parallel implementation of an algorithm requires the same amount of work as the sequential implementation, the parallel implementation is called <i>work efficient</i>. The advantage of this implementation of fastscan, based on contraction, is that it is work efficient, in contrast to the divide-and-conquer approach, which requries $O(n\log(n))$ work.

### Analysis of the Span of `scan` 

Assume that function `f` is constant time.


```python
    subproblem = [f(a[i], a[i+1]) for i in range(len(a))[::2]]
```



With infinite processors, this can be done in constant span.




```python
    ret = (
        [partial_output[i//2] if i%2==0 else
            f(partial_output[i//2], a[i-1])  
            for i in range(len(a))]
    )
```


With infinite processors, this can also be done in constant span. The line

```python
    partial_output = fastscan(f, id_, subproblem)
```



reduces problem in half with a recursive call, so the span satisfies the recursive equation

$$S(n) = S(n/2) + 1$$

Therefore, we have a span of
$$S(n) \in O(\log n)$$

## Runtime Analysis Results

$$W(n) \in O(n)$$

$$S(n) \in O(\log n)$$

- surprisingly the *same* work and span of `reduce`
- even though we're keeping track of output for all prefixes.



<br><br>
`scan` is a popular primitive in parallel programming, used to solve many problems, including:

- evaluating polynomials
- quicksort
- search for regular expressions (See homework)